# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

This non-commercial license of GraphLab Create for academic use is assigned to chenghsien.richo@gmail.com and will expire on July 18, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1505713159.log


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    features = ['constant'] + features # this is how you combine two lists
    
    feature_matrix = data_sframe[features].to_numpy()
    output_array = data_sframe[output].to_numpy()

    return feature_matrix, output_array

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [5]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return predictions

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [6]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [7]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [8]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [9]:
def normalize_features(features):
    norms = np.linalg.norm(features, axis=0)
    normalized_features = features / norms
    return normalized_features, norms

To test the function, run the following:

In [11]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [12]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [13]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [14]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [24]:
prediction = predict_output(simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [33]:
m, n = simple_feature_matrix.shape
ro = np.zeros(n)
for i in range(n):
    ro[i] = np.dot(
        simple_feature_matrix[:, i],
        output - prediction + weights[i] *simple_feature_matrix[:, i]
    )

***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [38]:
print('To make w[1]=0, l1_penalty is betweem {0}~{1}'.format(-ro[1]*2, ro[1]*2))
print('To make w[2]=0, l1_penalty is betweem {0}~{1}'.format(-ro[2]*2, ro[2]*2))

To make w[1]=0, l1_penalty is betweem -175878941.546~175878941.546
To make w[2]=0, l1_penalty is betweem -161933397.352~161933397.352


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [55]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.sum(
        feature_matrix[:, i] * (output - prediction + weights[i] * feature_matrix[:, i])
    )

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [56]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [145]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    converged = False
    weights = initial_weights
    old_weights = initial_weights
    while not converged:
        old_weights = weights.copy()
        for i in range(len(weights)):
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
        change = np.max(np.abs(weights - old_weights))
        print(change)
#         change = np.sum(np.abs(weights - old_weights))
#         print(weights)
#         print(old_weights)
#         print(change)
        if change < tolerance:
            converged = True
        
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [146]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [147]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [148]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

79400304.6581
9138168.3665
8194809.40937
6598905.03827
5522173.1943
4621129.81031
3867108.10625
3236118.81061
2708086.93955
2266213.1094
1896439.06265
1587000.40319
1328052.31095
1111356.328
930018.251241
778268.79269
651280.029038
545011.800818
456083.174344
381664.876995
319389.283634
267275.090397
223664.279321
187169.368346
156629.268444
131072.343461
109685.497423
91788.3058116
76811.3677894
64278.190664
53790.0302253
45013.2046616
37668.4784415
31522.1784089
26378.7594496
22074.5832053
18472.7119037
15458.5516701
12936.2067131
10825.4283904
9059.062091
7580.91070476
6343.94670622
5308.81596931
4442.58571219
3717.69673772
3111.08663489
2603.45604622
2178.65465672
1823.16737024
1525.68432524
1276.74107066
1068.41745344
894.085638061
748.19924143
626.116874106
523.954474062
438.461734928
366.918697163
307.049212277
256.948526993
215.022683281
179.937806487
150.577667948
126.008172072
105.447638076
88.2419305854
73.8436484337
61.7947089002
51.7117738649
43.2740537673
36.2131017707
30

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [149]:
def get_residual_sum_of_squares(output, predicted_output):
    RSS = sum((output - predicted_output) ** 2)
    return RSS

predictions = predict_output(normalized_simple_feature_matrix, weights)
print(weights)
print(get_residual_sum_of_squares(output, predictions))

[ 21624998.36636292  63157246.78545421         0.        ]
1.63049248148e+15


# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [150]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [151]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [152]:
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
(normalized_feature_matrix, norms) = normalize_features(feature_matrix) # normalize features

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [162]:
initial_weights = np.zeros(len(all_features) + 1).copy()
l1_penalty = 1e7
tolerance = 1.0

weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print(zip(weights1e7, ['constant'] + all_features))

71114625.7528
5024356.36869
5012324.91395
5000676.42086
4991243.17314
4984361.34335
4979504.92921
4973806.56525
2879222.99938
2706150.76622
2532295.64864
2368316.57782
2213124.35067
1552443.60013
507838.033159
438814.370591
371479.622608
313415.105581
264244.832139
222753.8608
187770.667306
158280.093357
133420.901606
112465.988296
94802.2134519
79912.6874505
67361.6926196
56781.942492
47863.835752
40346.3966195
34009.6378518
28668.1231465
24165.5406129
20370.1285268
17170.8195088
14473.990295
12200.7219837
10284.4905855
8669.22029242
7307.64249856
6159.912551
5192.44375235
4376.92449333
3689.48975358
3110.02272543
2621.56612397
2209.82595597
1862.75322643
1570.19134162
1323.57889082
1115.69910862
940.468686543
792.759753533
668.249816094
563.295266576
474.824758228
400.249326415
337.386626367
284.397069879
239.729992341
202.078274757
170.340092748
143.586673193
121.035115026
102.025478713
86.0014740676
72.4941812046
61.1083282754
51.5107243359
43.4205090664
36.6009337232
30.8524330631

***QUIZ QUESTION***

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [163]:
initial_weights = np.zeros(len(all_features) + 1).copy()
l1_penalty = 1e8
tolerance = 1.0

weights1e8 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print(zip(weights1e8, ['constant'] + all_features))

71114625.7528
0.0
[(71114625.752809376, 'constant'), (0.0, 'bedrooms'), (0.0, 'bathrooms'), (0.0, 'sqft_living'), (0.0, 'sqft_lot'), (0.0, 'floors'), (0.0, 'waterfront'), (0.0, 'view'), (0.0, 'condition'), (0.0, 'grade'), (0.0, 'sqft_above'), (0.0, 'sqft_basement'), (0.0, 'yr_built'), (0.0, 'yr_renovated')]


***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [155]:
l1_penalty = 1e4
tolerance = 1.0

weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print(zip(weights1e4, ['constant'] + all_features))

12531492.3845
8464015.94738
6818451.35869
5570165.21901
4559838.46759
3732056.69715
3053211.61353
2497517.43558
2043868.97325
1674717.70906
1375420.75692
1296621.8143
1257340.9361
1228271.24819
1206854.36471
1191059.40598
1179030.77017
1169777.43345
1162340.67589
1155980.52901
1150139.67966
1144406.32752
1138483.91805
1132166.57169
1125319.16778
1117861.24888
1109754.06282
1100990.17907
1091585.21244
1081571.26761
1070991.78405
1059897.51701
1048343.43631
1036386.3636
1024083.20068
1011489.62847
998659.178392
985642.596017
972487.432403
959237.810727
945934.326266
932614.046118
919310.581988
906054.214956
892872.055697
879109.302243
866038.266742
853191.44943
840521.781012
828028.403962
815717.920819
803598.206378
791676.769693
779960.222854
768454.139375
757163.05916
746090.546302
735239.263812
724611.051693
714002.910141
703393.002342
693038.299304
682960.589206
673155.135299
663513.600594
654148.681362
645045.446052
636191.878813
627577.229773
619191.96692
611027.657019
603076.79613

106507.713336
106360.212847
106213.300928
106066.975107
105921.232923
105776.071924
105631.489668
105487.483725
105344.051675
105201.191106
105058.899618
104917.174821
104776.014336
104635.41579
104495.376826
104355.895091
104216.968246
104078.593961
103940.769913
103803.493793
103666.763299
103530.57614
103394.930033
103259.822706
103125.251896
102991.21535
102857.710825
102724.736085
102592.288906
102460.367072
102328.968378
102198.090624
102067.731625
101937.889202
101808.561184
101679.745412
101551.439735
101423.64201
101296.350104
101169.561893
101043.275262
100917.488104
100792.198322
100667.403828
100543.10254
100419.292389
100295.971312
100173.137254
100050.788172
99928.9220278
99807.5367942
99686.6304515
99566.2009889
99446.2464039
99326.7647025
99207.753899
99089.2120161
98971.1370848
98853.5271444
98736.3802424
98619.6944346
98503.4677849
98387.6983653
98272.3842561
98157.5235455
98043.1143298
97929.1547133
97815.6428082
97702.5767349
97589.9546213
97477.7746035
97366.034825

71292.7899737
71274.6904769
71256.6372547
71238.6301159
71220.6688696
71202.7533261
71184.8832963
71167.0585919
71149.2790253
71131.54441
71113.85456
71096.2092901
71078.6084161
71061.0517542
71043.5391219
71026.070337
71008.6452182
70991.263585
70973.9252577
70956.6300573
70939.3778055
70922.1683249
70905.0014387
70887.8769709
70870.7947463
70853.7545903
70836.7563291
70819.7997897
70802.8847997
70786.0111876
70769.1787824
70752.387414
70735.6369129
70718.9271103
70702.2578384
70685.6289296
70669.0402175
70652.491536
70635.98272
70619.513605
70603.084027
70586.693823
70570.3428304
70554.0308875
70537.7578333
70521.5235072
70505.3277496
70489.1704013
70473.051304
70456.9702999
70440.9272319
70424.9219436
70408.9542793
70393.0240839
70377.1312028
70361.2754823
70345.4567693
70329.6749112
70313.9297561
70298.2211528
70282.5489508
70266.913
70251.3131512
70235.7492555
70220.221165
70204.7287321
70189.2718101
70173.8502526
70158.4639142
70143.1126498
70127.796315
70112.5147661
70097.267859

64539.7860546
64532.1749191
64524.5678411
64516.9648063
64509.3658009
64501.7708107
64494.1798219
64486.5928206
64479.009793
64471.4307253
64463.8556038
64456.2844149
64448.7171451
64441.1537806
64433.5943084
64426.0387146
64418.486986
64410.9391094
64403.3950717
64395.8548593
64388.3184593
64380.7858587
64373.2570443
64365.7320032
64358.2107226
64350.6931894
64343.179391
64335.6693146
64328.1629474
64320.6602768
64313.1612902
64305.6659752
64298.1743192
64290.6863096
64283.2019343
64275.7211809
64268.2440369
64260.7704902
64253.3005287
64245.8341403
64238.3713127
64230.9120342
64223.4562925
64216.0040758
64208.5553722
64201.1101699
64193.6684572
64186.2302223
64178.7954534
64171.3641391
64163.9362677
64156.5118275
64149.0908073
64141.6731955
64134.2589808
64126.8481517
64119.4406971
64112.0366057
64104.6358662
64097.2384675
64089.8443986
64082.4536481
64075.0662054
64067.6820593
64060.301199
64052.9236134
64045.5492921
64038.1782238
64030.810398
64023.4458041
64016.0844313
64008.72626

60363.7269374
60357.2710952
60350.8161896
60344.3622197
60337.9091842
60331.4570821
60325.0059121
60318.5556731
60312.106364
60305.6579835
60299.2105308
60292.7640045
60286.3184035
60279.8737268
60273.4299731
60266.9871415
60260.5452308
60254.1042401
60247.6641679
60241.2250134
60234.7867753
60228.3494528
60221.9130447
60215.4775498
60209.0429673
60202.6092958
60196.1765345
60189.7446821
60183.3137378
60176.8837005
60170.4545691
60164.0263425
60157.5990197
60151.1725998
60144.7470816
60138.3224642
60131.8987464
60125.4759273
60119.0540059
60112.6329813
60106.2128522
60099.793618
60093.3752773
60086.9578294
60080.5412731
60074.1256076
60067.7108318
60061.2969447
60054.8839455
60048.4718331
60042.0606064
60035.6502647
60029.2408069
60022.8322319
60016.4245391
60010.0177274
60003.6117958
59997.2067433
59990.8025691
59984.3992723
59977.9968517
59971.5953068
59965.1946362
59958.7948394
59952.3959153
59945.9978629
59939.6006815
59933.20437
59926.8089276
59920.4143533
59914.0206465
59907.6278

...

1.37407875061
1.37393307686
1.37378764153
1.37364196777
1.37349653244
1.37335085869
1.37320554256
1.37306010723
1.37291455269
1.37276911736
1.37262380123
1.3724783659
1.37233281136
1.37218737602
1.3720420599
1.37189662457
1.37175130844
1.37160587311
1.37146067619
1.37131536007
1.37117016315
1.37102472782
1.37087953091
1.37073421478
1.37058901787
1.37044382095
1.37029874325
1.37015354633
1.37000823021
1.3698631525
1.3697180748
1.36957299709
1.36942780018
1.36928272247
1.36913776398
1.36899256706
1.36884760857
1.36870253086
1.36855757236
1.36841249466
1.36826753616
1.36812269688
1.36797749996
1.36783266068
1.36768782139
1.36754286289
1.3673979044
1.36725318432
1.36710834503
1.36696338654
1.36681842804
1.36667382717
1.36652898788
1.3663841486
1.36623942852
1.36609470844
1.36594986916
1.36580514908
1.36566066742
1.36551594734
1.36537110806
1.36522650719
1.36508190632
1.36493730545
1.36479258537
1.36464810371
1.36450338364
1.36435902119
1.36421442032
1.36406981945
1.36392533779
1.3637806177

1.2898542881
1.28971767426
1.28958106041
1.28944444656
1.28930795193
1.28917109966
1.28903448582
1.28889799118
1.28876137733
1.28862500191
1.28848850727
1.28835189342
1.28821527958
1.28807902336
1.28794252872
1.28780591488
1.28766953945
1.28753304482
1.28739666939
1.28726041317
1.28712391853
1.28698754311
1.28685116768
1.28671479225
1.28657853603
1.28644227982
1.2863060236
1.28616964817
1.28603339195
1.28589713573
1.28576087952
1.2856246233
1.28548848629
1.28535223007
1.28521597385
1.28507995605
1.28494381905
1.28480756283
1.28467154503
1.28453540802
1.28439939022
1.28426325321
1.2841271162
1.2839910984
1.2838550806
1.28371918201
1.2835829258
1.283446908
1.28331112862
1.28317511082
1.28303909302
1.28290331364
1.28276729584
1.28263139725
1.28249549866
1.28235960007
1.28222370148
1.2820879221
1.28195214272
1.28181636333
1.28168046474
1.28154468536
1.28140878677
1.28127324581
1.28113734722
1.28100168705
1.28086590767
1.2807302475
1.28059458733
1.28045892715
1.28032314777
1.2801874876
1.28

1.2093821764
1.2092539072
1.20912587643
1.20899784565
1.20886969566
1.20874166489
1.20861339569
1.20848548412
1.20835745335
1.20822942257
1.208101511
1.20797336102
1.20784533024
1.20771741867
1.20758938789
1.20746147633
1.20733368397
1.2072057724
1.20707786083
1.20695006847
1.2068220377
1.20669424534
1.20656633377
1.2064384222
1.20631074905
1.2061830759
1.20605516434
1.20592725277
1.20579957962
1.20567190647
1.2055439949
1.20541632175
1.2052885294
1.20516097546
1.20503330231
1.20490550995
1.20477795601
1.20465028286
1.20452260971
1.20439505577
1.20426738262
1.20413982868
1.20401215553
1.2038847208
1.20375704765
1.20362961292
1.20350217819
1.20337462425
1.20324707031
1.20311963558
1.20299208164
1.20286464691
1.20273721218
1.20260965824
1.20248246193
1.20235490799
1.20222771168
1.20210015774
1.20197296143
1.2018455267
1.20171821117
1.20159089565
1.20146358013
1.20133626461
1.2012090683
1.20108175278
1.20095455647
1.20082712173
1.20070004463
1.20057272911
1.20044565201
1.20031833649
1.200

1.133690238
1.13357019424
1.13344991207
1.13332986832
1.13320982456
1.1330896616
1.13296973705
1.1328496933
1.13272964954
1.132609725
1.13248956203
1.13236963749
1.13224971294
1.13212966919
1.13200974464
1.1318898201
1.13176989555
1.13164997101
1.13153016567
1.13141024113
1.13129031658
1.13117063046
1.13105070591
1.13093078136
1.13081097603
1.13069105148
1.13057148457
1.13045156002
1.13033187389
1.13021194935
1.13009238243
1.1299725771
1.12985301018
1.12973320484
1.12961351871
1.12949383259
1.12937414646
1.12925446033
1.12913477421
1.1290153265
1.12889564037
1.12877595425
1.12865650654
1.12853682041
1.12841713428
1.12829768658
1.12817811966
1.12805855274
1.12793922424
1.12781953812
1.12770020962
1.1275806427
1.12746119499
1.12734186649
1.12722218037
1.12710285187
1.12698340416
1.12686407566
1.12674450874
1.12662541866
1.12650597095
1.12638640404
1.12626731396
1.12614774704
1.12602865696
1.12590932846
1.12578999996
1.12567079067
1.12555146217
1.12543213367
1.12531292439
1.1251937151
1.1

1.06329870224
1.06318604946
1.06307351589
1.0629607439
1.06284821033
1.06273555756
1.06262302399
1.06251049042
1.06239783764
1.06228518486
1.06217288971
1.06206011772
1.06194758415
1.06183516979
1.06172263622
1.06161022186
1.0614978075
1.06138515472
1.06127274036
1.061160326
1.06104791164
1.06093549728
1.06082308292
1.06071066856
1.0605982542
1.06048595905
1.0603736639
1.06026113033
1.06014895439
1.06003665924
1.05992436409
1.05981206894
1.05969965458
1.05958747864
1.05947518349
1.05936300755
1.05925071239
1.05913841724
1.05902636051
1.05891406536
1.05880188942
1.05868971348
1.05857753754
1.0584653616
1.05835318565
1.05824112892
1.05812907219
1.05801677704
1.05790483952
1.05779266357
1.05768060684
1.05756866932
1.05745649338
1.05734455585
1.05723249912
1.05712044239
1.05700838566
1.05689644814
1.05678451061
1.05667257309
1.05656051636
1.05644857883
1.05633664131
1.05622470379
1.05611300468
1.05600094795
1.05588901043
1.05577719212
1.05566525459
1.05555355549
1.05544173717
1.05532991886

***QUIZ QUESTION***

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [164]:
# (test_feature_matrix, output) = get_numpy_data(test_data, all_features, my_output)
# (normalized_test_feature_matrix, norms) = normalize_features(test_feature_matrix)
normalized_weights1e4 = weights1e4 / norms
normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms
print(normalized_weights1e4[3])
print(normalized_weights1e7[3])
print(normalized_weights1e8[3])

173.757192283
161.317456248
0.0


To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [157]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [165]:
test_predictions = predict_output(test_feature_matrix, normalized_weights1e4)

print(get_residual_sum_of_squares(test_output, test_predictions))

1.94495188688e+14


In [166]:
test_predictions = predict_output(test_feature_matrix, normalized_weights1e7)

print(get_residual_sum_of_squares(test_output, test_predictions))

2.75962079909e+14


In [167]:
test_predictions = predict_output(test_feature_matrix, normalized_weights1e8)

print(get_residual_sum_of_squares(test_output, test_predictions))

5.37166150034e+14


***QUIZ QUESTION***

Which model performed best on the test data?